<a href="https://colab.research.google.com/github/alper-oner/checkworthy-claims/blob/master/cosineSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install flair

In [0]:
!pip install nltk
import nltk
import re
import pandas as pd
import io
nltk.download('stopwords')
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')

def norm_tweet(tweet):
    tweet=tweet.replace("İ","i")
    tweet=tweet.replace("Ö","ö")
    tweet=tweet.replace("@","")
    tweet=tweet.replace("'", "")
    tweet=tweet.replace("’", "")
    tweet=tweet.replace('"', "")
    tweet=tweet.replace("#","")
    tweet=tweet.replace(",","")
    tweet=tweet.replace(".","")
    tweet=tweet.replace("!","")
    tweet=tweet.replace(";","")
    tweet=tweet.replace(":","")
    tweet=tweet.replace("/","")
    tweet=tweet.replace("“","")
    tweet=tweet.replace("”","")
    tweet=tweet.replace("?","")
    tweet=tweet.replace("-","")

    pattern = r"[{}]".format(",.;@#!") 
    tweet = re.sub(pattern, "", str(tweet))
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = WPT.tokenize(tweet)
    filtered_tokens = [token for token in tokens if token not in stop_word_list]
    tweet = ' '.join(filtered_tokens)
    return tweet

In [0]:
import urllib.request, json 
import ssl


context = ssl._create_unverified_context()

with urllib.request.urlopen("https://api.myjson.com/bins/1bgkqj",context=context) as url:
    dataClaim = json.loads(url.read().decode())

claimList=[]
for claim in dataClaim["claims"]:
    #print(claim["text"])
    claimList.append(claim["text"])

    

In [0]:
import urllib.request, json 
import ssl

context = ssl._create_unverified_context()

# test_balanced : https://api.myjson.com/bins/qnijh
# test_unbalanced : https://api.myjson.com/bins/7da99
with urllib.request.urlopen("https://api.myjson.com/bins/7da99",context=context) as url:
    dataTweet = json.loads(url.read().decode())

tweetList=[]
for tweet in dataTweet:
    #print(tweet["tweet"])
    tweetList.append(tweet["tweet"])


In [0]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence, CharacterEmbeddings

# initialize the word embeddings
tr_embedding = WordEmbeddings('tr')
char_embedding = CharacterEmbeddings()

# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([tr_embedding,char_embedding])
claimTensors=[]

for claim in claimList:
    #print(norm_tweet(claim))
    sentence = Sentence(norm_tweet(claim))
    document_embeddings.embed(sentence)
    claimTensors.append(sentence.get_embedding().data)  


In [0]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence, CharacterEmbeddings

tweetTensors=[]
for tweet in tweetList:
    #print(norm_tweet(tweet))
    sentence = Sentence(norm_tweet(tweet))
    document_embeddings.embed(sentence)
    tweetTensors.append(sentence.get_embedding().data)

In [0]:
from scipy import spatial
check_worthy_tweet=[]
for tweeetIndex in range(len(tweetTensors)):
  for claimIndex in range(len(claimTensors)):
    result = 1 - spatial.distance.cosine(tweetTensors[tweeetIndex],claimTensors[claimIndex])
    if result > 0.80:
      if tweetList[tweeetIndex] not in check_worthy_tweet:
        check_worthy_tweet.append(tweetList[tweeetIndex])


In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


actual = []
predicted = []

for tweet in dataTweet:
    if tweet["has_checkworthy_claim"] == 1:
      actual.append(1)
    else:
      actual.append(0)
      
for tweet in dataTweet:
    if tweet["tweet"] in check_worthy_tweet:
      predicted.append(1)
    else:
      predicted.append(0)


results = confusion_matrix(actual, predicted) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, predicted) )
print ('Report : ')
print (classification_report(actual, predicted))

In [0]:
!pip install pandas_ml
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(actual, predicted)
cm.print_stats()